# 🚀 Analyse 1: Diachrone Frequenzdiagramme

Dieses **Notebook** kann auf unterschiedlichen Levels erarbeitet werden (siehe Abschnitt ["Technische Voraussetzungen"](../markdown/introduction_requirements)): 
1. Book-Only Mode
2. Cloud Mode: Dafür auf 🚀 klicken und z.B. in Colab ausführen.
3. Local Mode: Dafür auf Herunterladen ↓ klicken und ".ipynb" wählen. 

## Übersicht 
Im Folgenden werden die annotierten Dateien (CSV-Format) analysiert. Unser Ziel ist es, die Wort-/Lemma-Häufigkeiten einer vordefinierten Wortgruppe für die Monate des Jahres 1918 zu plotten und zu sehen, ob sie mit den Wellen der Grippepandemie korrelieren.
Dafür werden folgendene Schritte durchgeführt:
1. Einlesen des Korpus, der Metadaten und der Grippe-Wortliste
2. Extraktion der Worthäufigkeiten und Plotten der Worthäufigkeiten
3. Diskussion der Ergebnisse

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
  
<b>Voraussetzungen zur Ausführung des Jupyter Notebooks</b>
<ol>
<li> Installieren der Bibliotheken </li>
<li> Pfad zu den Daten setzen</li>
<li> Laden der Daten (z.B. über den Command `wget` (s.u.))</li>
</ol>
Zum Testen: Ausführen der Zelle "load libraries" und der Sektion "Einlesen der Daten". </br>
Alle Zellen, die mit 🚀 gekennzeichnet sind, werden nur bei der Ausführung des Noteboos in Colab / JupyterHub bzw. lokal ausgeführt. 
</details>

In [ ]:
#  🚀 Install libraries 
! pip install pandas bokeh 

In [1]:
import re
import requests
from pathlib import Path
import pandas as pd

## for interactivity in jupyter books
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, TextInput, Div, RadioButtonGroup
# Ensure Bokeh output is displayed in the notebook
output_notebook()

Loading BokehJS ...

## 1. Einlesen der Daten, Metadaten und der Grippe-Wortliste
Um eine/mehrere Dateien mit Python bearbeiten zu können, müssen die Dateien zuerst ausgewählt werden, d.h der [Pfad](https://en.wikipedia.org/wiki/Path_(computing)) zu den Dateien wird gesetzt, und dann eingelesen werden. 

### 1.1 Einlesen des Korpus (CSV-Dateien)

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Zuerst wird der Ordner angelegt, in dem die CSV-Dateien gespeichert werden. Der Einfachheit halber wird die gleich Datenablagestruktur wie in dem <a href="https://github.com/dh-network/quadriga/tree/main">GitHub Repository</a>, in dem die Daten gespeichert sind, vorausgesetzt. </br>
Danach werden alle CSV-Dateien im Korpus heruntergeladen und gespeichert. Dafür sind folgende Schritte nötig:
<ol>
    <li>Es wird eine Liste erstellt, die die URLs zu den einzelnen CSV-Dateien beinhaltet.</li>
    <li>Die Liste wird als txt-Datei gespeichert.</li>
    <li>Alle Dateien aus der Liste werden heruntergeladen und in dem Ordner <i>../data/csv</i> gespeichert.</li>
</ol>
Sollten die Dateien schon an einem anderen Ort vorhanden sein, können die Dateipfade zu den Ordnern angepasst werden. </br>
</details>

In [ ]:
# 🚀 Create data directory path
corpus_dir = Path("../data/csv")
if not corpus_dir.exists():
    corpus_dir.mkdir()

In [ ]:
# 🚀 Create download list 
github_api_txt_dir_path = "https://api.github.com/repos/dh-network/quadriga/contents/data/csv"
txt_dir_info = requests.get(github_api_txt_dir_path).json()
url_list = [entry["download_url"] for entry in txt_dir_info]

# 🚀 Write download list as txt file
url_list_path = Path("github_csv_file_urls.txt")
with url_list_path.open('w') as output_txt:
    output_txt.write("\n".join(url_list))

In [ ]:
# ⚠️ Only execute, if you haven't downloaded the files yet!
# 🚀 Download all csv files – this step will take a while (ca. 7 minutes)
! wget -i github_csv_file_urls.txt -P ../data/csv

Setzen des Pfads:

In [2]:
# set the path to csv files to be processed
csv_dir = Path(r"../data/csv")

Einlesen der CSV-Dateien

In [3]:
# Create dictionary to save the corpus data (filenames and tables)
corpus_annotations = {}

# Iterate over csv files 
for file in csv_dir.iterdir():
    # check if the entry is a file, not a directory
    if file.is_file():
        # check if the file has the correct suffix csv
        if file.suffix == '.csv':
            # read the csv table to a data frame
            data = pd.read_csv(file) 
            # save the data frame to the dictionary, key=filename (without suffix), value=dataframe
            corpus_annotations[file.with_suffix("").name] = data

Wie viele Dateien wurden eingelesen?

In [4]:
len(corpus_annotations)

1323

Wie sieht der Anfang der ersten Datei aus?

In [5]:
corpus_annotations[list(corpus_annotations.keys())[0]].head()

,Token,Lemma
0,48,48
1,",",--
2,warde,warde
3,],]
4,üvne,üvne


### 1.2 Einlesen der Metadaten

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Zuerst wird der Ordner angelegt, in dem die Metadaten-Datei gespeichert wird. Wieder wird die gleich Datenablagestruktur wie in dem <a href="https://github.com/dh-network/quadriga/tree/main">GitHub Repository</a> vorausgesetzt. </br>
Der Text wird aus GitHub heruntergeladen und in dem Ordner <i>../data/metadata/</i> abgespeichert. </br>
Der Pfad kann in der Variable <i>metadata_path</i> angepasst werden. Die einzulesende Datei muss die Endung `.csv` haben. </br>
</details>

In [ ]:
# 🚀 Create metadata directory path
metadata_dir = Path("../data/metadata")
if not metadata_dir.exists():
    metadata_dir.mkdir()

In [ ]:
# 🚀 Load the metadata file from GitHub 
! wget https://raw.githubusercontent.com/dh-network/quadriga/refs/heads/main/data/metadata/QUADRIGA_FS-Text-01_Data01_Corpus-Table.csv -P ../data/metadata

In [6]:
# set path to metadata file
metadata_path = '../data/metadata/QUADRIGA_FS-Text-01_Data01_Corpus-Table.csv'

# read metadata file to pandas dataframe
corpus_metadata = pd.read_csv(metadata_path, sep=';')
corpus_metadata['DC.date'] = pd.to_datetime(corpus_metadata['DC.date'])
#corpus_metadata = corpus_metadata.set_index('DC.identifier')

Wie sieht die Metadaten-Datei aus? (erste fünf Zeilen)

In [7]:
corpus_metadata.head()

,DC.identifier,DC.publisher,DC.date,DC.source
0,SNP2719372X-19180101-0-0-0-0,Berliner Morgenpost,1918-01-01,https://content.staatsbibliothek-berlin.de/zef...
1,SNP2719372X-19180102-0-0-0-0,Berliner Morgenpost,1918-01-02,https://content.staatsbibliothek-berlin.de/zef...
2,SNP2719372X-19180103-0-0-0-0,Berliner Morgenpost,1918-01-03,https://content.staatsbibliothek-berlin.de/zef...
3,SNP2719372X-19180104-0-0-0-0,Berliner Morgenpost,1918-01-04,https://content.staatsbibliothek-berlin.de/zef...
4,SNP2719372X-19180105-0-0-0-0,Berliner Morgenpost,1918-01-05,https://content.staatsbibliothek-berlin.de/zef...


### 1.3 Einlesen der Wortliste 

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Parallel zur Metadaten-Datei wird ein Ordner für die Wortlisten-Datein angelegt, die Datei wird aus GitHub geladen und in dem erstellten Ordner abgelegt.
</details>

In [ ]:
# 🚀 Create word list directory path
wordlist_dir = Path("../data/wordlist")
if not wordlist_dir.exists():
    wordlist_dir.mkdir()

In [ ]:
# 🚀 Load the wordlist file from GitHub 
! wget https://raw.githubusercontent.com/dh-network/quadriga/refs/heads/main/data/wordlist/grippe.txt -P ../data/wordlist

In [9]:
path_to_wordlist = Path("../data/wordlist/grippe.txt")
word_list = path_to_wordlist.read_text().split("\n")

Wie sieht die Wortliste aus?

In [10]:
word_list

['Influenza',
 'Grippe',
 'Grippeepidemie',
 'Grippewelle',
 'Grippekranke',
 'Grippepandemie',
 'Lungenentzündung',
 'Krankheitswelle',
 'Seuchenzug',
 'Krankheitsausbruch',
 'Fieberanfall',
 'Schüttelfrost',
 'Atemnot',
 'Körpererschöpfung',
 'Genesungszeit',
 'Ansteckungsgefahr',
 'Seuchenschutz',
 'Desinfektionsmittel',
 'Schutzmaske',
 'Krankenstation',
 'Isolationsstation',
 'Sanitätsdienst',
 'Krankheitsverlauf',
 'Todesopfer',
 'Krankheitssymptom',
 'Erkrankungsfall',
 'Lungeninfektion',
 '']

## 2. Suche nach einem Lemma und plotte die Häufigkeit

1. Datum zu den Annotationen hinzufügen
2. Annotationen in einer Datenstruktur (einem DataFrame) speichern
3. Lemmata suchen und nach Zeitabschnitt gruppieren
4. Häufigkeiten plotten

### 2.1 Datum zu den Annotationen hinzufügen

In [11]:
def add_date_to_corpus_annotations(corpus_metadata, corpus_annotated):
    for identifier, df in corpus_annotated.items():
        if identifier in corpus_metadata["DC.identifier"].values:
            df["date"] = corpus_metadata[corpus_metadata["DC.identifier"] == identifier]["DC.date"].item()

In [12]:
add_date_to_corpus_annotations(corpus_metadata, corpus_annotations)

### 2.2 Annotationen in einer Datenstruktur (einem DataFrame) speichern

In [19]:
corpus_annotations_merged = pd.concat(corpus_annotations.values())

In [14]:
def search_and_plot(merged_df, search_terms=word_list, absolute_freqs=True):
    result = merged_df.query(f'Lemma.isin({search_terms})')
        
    freqs_month = result.groupby(pd.PeriodIndex(result['date'], freq="M")).count().Lemma
    freqs_week = result.groupby(pd.PeriodIndex(result['date'], freq="W-MON")).count().Lemma
    freqs_days = result.groupby(pd.PeriodIndex(result['date'], freq="D")).count().Lemma
    
    plot_with_js(freqs_month, freqs_week, freqs_days, search_terms)

In [15]:
def plot_with_js(freqs_month, freqs_week, freqs_day, words):
    # Prepare data sources
    daily_source = ColumnDataSource(data=dict(x=freqs_day.index, y=list(freqs_day)))
    weekly_source = ColumnDataSource(data=dict(x=freqs_week.index, y=list(freqs_week)))
    monthly_source = ColumnDataSource(data=dict(x=freqs_month.index, y=list(freqs_month)))
    starter_source = ColumnDataSource(data=dict(x=freqs_month.index, y=list(freqs_month)))

    # Create a plot
    p = figure(title=f"Frequency of words {words}", x_axis_type="datetime", x_axis_label='Time', 
               y_axis_label='Frequency', width=700, height=400)
    line = p.line('x', 'y', source=starter_source, line_width=2, color='blue')

    # Callback to update the data based on selected mode
    callback = CustomJS(args=dict(line=line, daily_source=daily_source, 
                                  weekly_source=weekly_source, monthly_source=monthly_source),
                        code="""
        const mode = cb_obj.active;
        if (mode === 2) {
            line.data_source.data = daily_source.data;
        } else if (mode === 1) {
            line.data_source.data = weekly_source.data;
        } else if (mode === 0) {
            line.data_source.data = monthly_source.data;
        }
        line.data_source.change.emit();
    """)

    # RadioButtonGroup to select mode
    radio_button_group = RadioButtonGroup(labels=["Monthly", "Weekly", "Daily"], active=0)
    radio_button_group.js_on_change('active', callback)

    # Layout the RadioButtonGroup and plot
    layout = column(radio_button_group, p)
    show(layout)

In [16]:
search_and_plot(corpus_annotations_merged)

### Worteingabe für die Suche (für Cloud Mode und Local Mode) 

In [20]:
search_terms = TextInput(value='Grippe, Krankheit', 
                                 title="Geben Sie die zu suchenden Wörter ein und trennen Sie sie durch Kommas, wenn es mehrere sind:") #input('Insert words to search, split by comma if more than one: ')

search_terms_str = search_terms.value.strip()

# JavaScript callback to update the in Jupyter Book
rewrite_var_after_input = CustomJS(args=dict(text_input=search_terms), code="""
    var word = text_input.value.trim();
    console.log('Input value:', word);
    function sendToPython(){
    var kernel = IPython.notebook.kernel;
    kernel.execute("search_terms_str = '" + word + "'");
    }
    sendToPython();
""")

search_terms.js_on_change('value', rewrite_var_after_input)

# Layout and display
layout = column(search_terms)

show(layout)

In [24]:
search_terms =  [x.strip() for x in search_terms_str.split(',')]
search_and_plot(corpus_annotations_merged, search_terms=search_terms)

## 3. Diskussion des Zwischenergebnisses

Ist dieses Ergebnis sinnvoll und spiegelt es tatsächlich etwas wider? Eine Möglichkeit, dies zu überprüfen, besteht darin, unser Diagramm mit den tatsächlichen Daten über die Intensität der Pandemie zu vergleichen.

In (Taubenberger, J. K., & Morens, D. M. (2006). 1918 Influenza: the Mother of All Pandemics. Emerging Infectious Diseases, 12(1), 15-22. https://doi.org/10.3201/eid1201.050979) wird festgestellt, dass 'The first pandemic influenza wave appeared in the spring of 1918, followed in rapid succession by much more fatal second and third waves in the fall and winter of 1918–1919, respectively'('Die erste Pandemie-Influenza-Welle im Frühjahr 1918 auftrat, gefolgt von weitaus tödlicheren zweiten und dritten Wellen im Herbst und Winter 1918–1919'). Sie ergänzen diese Aussage auch mit einem Diagramm aus einem früheren Papier (Jordan E. (1927). Epidemic influenza: a survey. Chicago: American Medical Association):

<!-- ## 3. Discussion of the intermediate result 

Is this result meaningful and does it actually reflect something? One way to check that is to compare our plot with the actual data about the intensity of the pandemic. 

In (Taubenberger, J. K., & Morens, D. M. (2006). 1918 Influenza: the Mother of All Pandemics. Emerging Infectious Diseases, 12(1), 15-22. https://doi.org/10.3201/eid1201.050979) it is stated that 'The first pandemic influenza wave appeared in the spring of 1918, followed in rapid succession by much more fatal second and third waves in the fall and winter of 1918–1919, respectively'. They also supplement this statement with a plot from an earlier paper (Jordan  E. (1927). Epidemic influenza: a survey. Chicago: American Medical Association): --> 

<img src="https://wwwnc.cdc.gov/eid/images/05-0979-F1.gif">

Unsere zwei Wellen von Erwähnungen des Wortes 'Grippe' scheinen den Sterblichkeitszahlen zu entsprechen, was darauf hindeuten könnte, dass die Methode, obwohl sehr einfach, funktioniert und dass historische Ereignisse manchmal in Wortfrequenzzählungen reflektiert werden können... Die dritte Welle scheint nicht reproduziert zu werden, was eine weitere Untersuchung erfordert. Eine Hypothese könnte sein, dass, ähnlich wie bei der COVID-Pandemie, neue Krankheitswellen irgendwann aufhören, die Aufmerksamkeit der Öffentlichkeit zu erregen. Beispielsweise waren die COVID-Wellen im Jahr 2021 stärker als die im Jahr 2020, aber die Berichterstattung in den Nachrichten nahm bereits ab. Dies könnte besonders für Anfang 1919 zutreffen, als nach dem Verlust des Krieges und der Revolution von 1918 Grippetodesfälle kein Nachrichtenthema mehr waren.
<!-- Our two waves of mentions of the word 'Grippe' seem to correspond to the mortality figures, which could indicate that the method, albeit very simple, works and that historical events can sometimes be reflected in word frequency counts... The third wave does not seem to rerpoduce, which calls for further investigation. One hypothesis could be that, like with the COVID pandemic, at some point new waves of illness stop attracting public's attention. E.g. the 2021 covid waves were stronger than the 2020, but the news coverage was already waning. This coul be especially true for early 1919, when after the loss of the war and the 1918 revolution grippe deaths were not a news topic anymore. -->